# Here we want to :
## 1. Exclude stations which are not in Australia 
## 2. Exclude no-values from the Dataset
## 3. Convert all precipitation units to mm/day

In [1]:
import pandas as pd
import h5py 
import numpy as np

def Dict_to_hdf(Dict , of ):
    HDF = h5py.File(of, 'w')
    for k in Dict :    
        test = HDF.create_dataset( k , data = Dict[k] )
    HDF.close()   

def nc_to_pd(nc):
    try:
        Keys = list(nc.keys())
    except:
        Keys = list(nc.variables)
    Panda = {}
    for k in Keys:
        try:
            Panda[k]= np.array(nc[k] )
            PD = pd.DataFrame( Panda )
        except:
            Panda[k] = np.array(nc[k]).T[0]
            PD = pd.DataFrame( Panda )
            pass
    return PD , Panda



In [4]:
fp = '../dataset/noaa_obs_2014-2024.nc4_resample-0016.hdf5'
hdf = h5py.File( fp )
df , Dict = nc_to_pd(hdf)


In [5]:
# insert a list of stations and the key you want to exclude them from the dataset

Features = { 'STATION' : [94394099999,
97372599999,
94393099999,
91574099999,
94293099999,
94995099999,
94298099999,
97378099999,
97380099999,
94299099999] }

In [6]:
qdf = df
for Key in Features :
    for i in range(len(Features[Key])):
        qdf = qdf[qdf[Key] != Features[Key][i]]




In [9]:
# add name to df 
ex = pd.read_excel('D:\Projects\precipitation_AUS\stations\stations_noaa_n_100.xlsx')

In [19]:
d_ex = {}
for i in range(len(ex)):
    d_ex[ex['STATION'][i]] = ex['NAME'][i][2:-5]
print(d_ex)

{94672099999: 'ADELAIDE INTERNATIONAL', 95677099999: 'ADELAIDE PARAFIELD', 94210099999: 'ADELE ISLAND', 94846099999: 'AIREYS INLET', 94802099999: 'ALBANY AIRPORT', 95896099999: 'ALBURY AEROPORT', 94896099999: 'ALBURY', 94326099999: 'ALICE SPRINGS', 94595199999: 'AMBERLEY', 94553099999: 'APPLETHORPE', 94217099999: 'ARGYLE AERODROME', 94676099999: 'ARKAROOLA', 95323099999: 'ARLTUNGA', 95773099999: 'ARMIDALE AIRPORT AWS', 94690099999: 'AUSTIN PLAINS', 94854099999: 'AVALON AIRPORT', 94895599999: 'AVALON', 95296099999: 'AYR ALVA BEACH', 95295099999: 'AYR DPI RESEARCH STATION', 94752099999: 'BADGERY S CREEK AIRPORT', 94603099999: 'BADGINGARRA RESEARCH STATION', 94912099999: 'BAIRNSDALE AIRPORT', 94852099999: 'BALLARAT AERODROME', 95487099999: 'BALLERA GAS FIELD', 94596099999: 'BALLINA AIRPORT AWS', 94568099999: 'BAMAGA INJINOO', 95304099999: 'BARROW ISLAND AIRPORT', 94125099999: 'BATCHELOR AERO', 94729099999: 'BATHURST AIRPORT AWS', 95575099999: 'BEAUDESERT DRUMLEY ST', 95566099999: 'BEERBUR

In [30]:
NAMES = []
stations = np.array(qdf['STATION'])
for i in range(len(qdf)):
    st = stations[i]
    NAMES.append(d_ex[st])
qdf['NAME'] = NAMES



In [34]:
qdf['ERA5_tp'] = qdf['ERA5_tp'] *24 * 1000
qdf['PRCP']   =  qdf['PRCP'] *24

In [35]:
qdf = qdf[qdf['CAL_P']>=0][qdf['PDIR']>=0][qdf['ERA5_tp'] >= 0 ][qdf['CAL_P(-1)']>=0]

C:\Users\S4055367\AppData\Local\Temp\ipykernel_2248\1708832308.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  qdf = qdf[qdf['CAL_P']>=0][qdf['PDIR']>=0][qdf['ERA5_tp'] >= 0 ][qdf['CAL_P(-1)']>=0]
C:\Users\S4055367\AppData\Local\Temp\ipykernel_2248\1708832308.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  qdf = qdf[qdf['CAL_P']>=0][qdf['PDIR']>=0][qdf['ERA5_tp'] >= 0 ][qdf['CAL_P(-1)']>=0]
C:\Users\S4055367\AppData\Local\Temp\ipykernel_2248\1708832308.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  qdf = qdf[qdf['CAL_P']>=0][qdf['PDIR']>=0][qdf['ERA5_tp'] >= 0 ][qdf['CAL_P(-1)']>=0]


In [36]:
len(qdf)

1574809

In [37]:
of = '../dataset/noaa_obs_2014-2024.nc4_resample-prepared-0017.hdf5'
Dict_to_hdf(Dict , of )